In [3]:
import cv2,os
data_path='dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
label_dict=dict(zip(categories,labels))

print(categories)
print(labels)
print(label_dict)


['without_mask', 'with_mask']
[0, 1]
{'without_mask': 0, 'with_mask': 1}


In [4]:
import numpy as np
img_size=100
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print('exception',e)

    


In [5]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)


Using TensorFlow backend.


In [6]:
new_target.shape

(1376, 2)

In [30]:
np.save('data',data)
np.save('target',new_target)


In [31]:
data=np.load('data.npy')
target=np.load('target.npy')


In [32]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [33]:
model=Sequential()
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#the first layer of CNN ollowed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#the second convolution layer followed by Relu and MaxPooling layers


model.add(Flatten())
model.add(Dropout(0.5))
#flatten layer to stack the output concolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])





In [35]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)


In [36]:
checkpoint=ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Train on 990 samples, validate on 248 samples
Epoch 1/20
990/990 [==============================] - 65s 66ms/step - loss: 0.7011 - acc: 0.5263 - val_loss: 0.6411 - val_acc: 0.5847
Epoch 2/20
990/990 [==============================] - 59s 59ms/step - loss: 0.5963 - acc: 0.7020 - val_loss: 0.5430 - val_acc: 0.7621
Epoch 3/20
990/990 [==============================] - 59s 59ms/step - loss: 0.4712 - acc: 0.7848 - val_loss: 0.4883 - val_acc: 0.7742
Epoch 4/20
990/990 [==============================] - 58s 59ms/step - loss: 0.3579 - acc: 0.8667 - val_loss: 0.2676 - val_acc: 0.9073
Epoch 5/20
990/990 [==============================] - 58s 59ms/step - loss: 0.2387 - acc: 0.9040 - val_loss: 0.2312 - val_acc: 0.9073
Epoch 6/20
990/990 [==============================] - 59s 59ms/step - loss: 0.2014 - acc: 0.9242 - val_loss: 0.2757 - val_acc: 0.8952
Epoch 7/20
990/990 [==============================] - 59s 60ms/step - loss: 0.1708 - acc: 0.9515 - val_loss: 0.1790 - val_acc: 0.9234
Epoch 8/20
990/9

In [37]:
print(model.evaluate(test_data,test_target))

138/138 [==============================] - 2s 16ms/step
[0.049911201243167336, 0.9782608695652174]
